## 0.0 imports

In [ ]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

## 1.0 api

In [ ]:
# CONFIGURAÇÃO DO DRIVER
service = Service('./chromedriver') 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# ACESSAR O SITE
driver.get("https://localize.assertivasolucoes.com.br")
input("🟡 Faça login e vá para a tela de CONSULTA POR CNPJ. Depois pressione ENTER aqui...")

# LER CNPJs DO EXCEL
df = pd.read_excel("Listagem.Xlsx", sheet_name="Listagem Total")
cnpjs_unicos = df.iloc[:, 1].dropna().astype(str).str.replace(r'\D', '', regex=True).unique().tolist()

# DICT DE RESULTADOS
resultados = {}

# LOOP DE CONSULTAS
for cnpj in cnpjs_unicos:
    try:
        print(f"\n🔍 Consultando CNPJ: {cnpj}")

        if cnpj in resultados:
            print(f"⏭️ CNPJ {cnpj} já processado, pulando...")
            continue

        # INSERE O CNPJ NO CAMPO
        campo = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.NAME, "currentSimpleFilterValue")))
        campo.clear()
        campo.send_keys(cnpj)

        # CLICA EM CONSULTAR
        WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@type="submit"]'))
        ).click()

        # AGUARDA O RESULTADO APARECER
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'MuiGrid-item') and contains(., 'CNPJ:')]"))
        )
        time.sleep(1.5) 

        # COLETAR DADOS
        dados = {}

        # Nome Fantasia
        try:
            full_text = driver.find_element(
                By.XPATH, "//strong[text()='Nome fantasia:']/parent::div").text
            nome_fantasia = full_text.replace("Nome fantasia:", "").strip()
            dados["Nome Fantasia"] = nome_fantasia
        except:
            dados["Nome Fantasia"] = "Dado não localizado"

        # Situação
        try:
            dados["Situação"] = driver.find_element(
                By.XPATH, "//div[contains(@class, 'MuiChip-root')]//span[contains(@class, 'MuiChip-label')]"
            ).text.strip()
        except:
            dados["Situação"] = "Dado não localizado"

        # Telefones
        try:
            telefones = []
            if "Não localizamos nenhum telefone" not in driver.page_source:
                elementos = driver.find_elements(
                    By.XPATH, "//div[contains(@class, 'phone')]//h6[contains(@class, 'number')]"
                )
                telefones = [tel.text.strip() for tel in elementos if tel.text.strip()]
            dados["Telefones"] = ', '.join(telefones) if telefones else "Não encontrado"
        except:
            dados["Telefones"] = "Erro na captura"

        # Endereço
        try:
            endereco = driver.find_element(
                By.XPATH, "//h6[contains(@class,'MuiTypography-subtitle1')]"
            ).text.strip()
            dados["Endereço"] = endereco
        except:
            dados["Endereço"] = "Endereço não localizado"

        # Sócios
        try:
            socios_elementos = driver.find_elements(
                By.XPATH, "//div[span and p[contains(text(),'Data de entrada:')]]/span"
            )
            socios = [el.text.strip() for el in socios_elementos]
            dados["Sócios"] = ', '.join(socios) if socios else "Nenhum sócio encontrado"
        except NoSuchElementException:
            dados["Sócios"] = "Nenhum sócio encontrado"
        except Exception as e:
            dados["Sócios"] = f"Erro na captura: {e}"
            
        # Email
        try:
            element = driver.find_element(
                By.XPATH, "//div[contains(@class, 'MuiBox-root') and contains(., '@')]"
            )
            texto = element.text
            email_match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", texto)
            dados["E-mail"] = email_match.group(0) if email_match else "E-mail não localizado"
        except NoSuchElementException:
            dados["E-mail"] = "E-mail não localizado"
        except Exception as e:
            dados["E-mail"] = f"Erro na captura: {e}"

        # ARMAZENAR RESULTADO
        resultados[cnpj] = dados
        print(f"✅ Dados capturados: {dados}")

        # VOLTAR PARA TELA DE CONSULTA
        driver.get("https://localize.assertivasolucoes.com.br/consulta/cnpj")
        time.sleep(3)

    except Exception as e:
        print(f"❌ Erro ao processar {cnpj}: {e}")
        resultados[cnpj] = {"Erro": str(e)}
        driver.get("https://localize.assertivasolucoes.com.br/consulta/cnpj")
        time.sleep(3)

# EXPORTAR PARA EXCEL 
pd.DataFrame.from_dict(resultados, orient='index').to_excel("resultados_assertiva_final.xlsx")
print(f"\n✅ Processo finalizado! Total de CNPJs processados: {len(resultados)}")
driver.quit()

## 2.0 organizando planilha

In [3]:
dfnew = pd.read_excel('resultados_assertiva_final.xlsx')
dfnew.head()

,Unnamed: 0,CNPJ,Nome Fantasia,Situação,Telefones,Endereço
0,42914169000105,42.914.169/0001-05,Dado não localizado,Ativa,(21) 3352-2222,"EST DO SAPE, 170, AP 101, TURIACU, RIO DE JANE..."
1,42921498000175,42.921.498/0001-75,Dado não localizado,Ativa,"(11) 98214-6897, (11) 4301-0143","R JULIO VERNE, 13, JARDIM HIPICO, SAO PAULO - ..."
2,42902088000187,42.902.088/0001-87,ATLAS ASSESSORIA LINGUISTICA,Ativa,(55) 99998-4967,"R MARECHAL FLORIANO PEIXOTO, 1279, AP 203 BLCO..."
3,42904264000110,42.904.264/0001-10,AMARINGLES,Ativa,(51) 99304-1190,"R FLORES DA CUNHA, 364, AP 402, CENTRO, SAO LE..."
4,42898012000125,42.898.012/0001-25,Dado não localizado,Ativa,"(11) 7789-7160, (11) 2669-0719","R BALACLAVA, 632, AP 3, JARDIM SANTO ALBERTO, ..."


### 2.1 split telefones

In [4]:
telefones_split = dfnew['Telefones'].str.split(', ', expand=True)
telefones_split

,0,1,2,3,4
0,(21) 3352-2222,None,None,None,None
1,(11) 98214-6897,(11) 4301-0143,None,None,None
2,(55) 99998-4967,None,None,None,None
3,(51) 99304-1190,None,None,None,None
4,(11) 7789-7160,(11) 2669-0719,None,None,None
...,...,...,...,...,...
320,(11) 98626-4176,None,None,None,None
321,(27) 99620-1995,(31) 98772-0781,None,None,None
322,(31) 99129-3467,None,None,None,None
323,(21) 98563-3469,(24) 3322-7461,None,None,None


In [5]:
telefones_split.columns = [f"Telefone_{i+1}" for i in range(telefones_split.shape[1])]
telefones_split

,Telefone_1,Telefone_2,Telefone_3,Telefone_4,Telefone_5
0,(21) 3352-2222,None,None,None,None
1,(11) 98214-6897,(11) 4301-0143,None,None,None
2,(55) 99998-4967,None,None,None,None
3,(51) 99304-1190,None,None,None,None
4,(11) 7789-7160,(11) 2669-0719,None,None,None
...,...,...,...,...,...
320,(11) 98626-4176,None,None,None,None
321,(27) 99620-1995,(31) 98772-0781,None,None,None
322,(31) 99129-3467,None,None,None,None
323,(21) 98563-3469,(24) 3322-7461,None,None,None


### 2.3 concat final

In [7]:
df_final = pd.concat([dfnew.drop(columns=["Telefones"]), telefones_split], axis=1)
df_final.head()

,Unnamed: 0,CNPJ,Nome Fantasia,Situação,Endereço,Telefone_1,Telefone_2,Telefone_3,Telefone_4,Telefone_5
0,42914169000105,42.914.169/0001-05,Dado não localizado,Ativa,"EST DO SAPE, 170, AP 101, TURIACU, RIO DE JANE...",(21) 3352-2222,None,None,None,None
1,42921498000175,42.921.498/0001-75,Dado não localizado,Ativa,"R JULIO VERNE, 13, JARDIM HIPICO, SAO PAULO - ...",(11) 98214-6897,(11) 4301-0143,None,None,None
2,42902088000187,42.902.088/0001-87,ATLAS ASSESSORIA LINGUISTICA,Ativa,"R MARECHAL FLORIANO PEIXOTO, 1279, AP 203 BLCO...",(55) 99998-4967,None,None,None,None
3,42904264000110,42.904.264/0001-10,AMARINGLES,Ativa,"R FLORES DA CUNHA, 364, AP 402, CENTRO, SAO LE...",(51) 99304-1190,None,None,None,None
4,42898012000125,42.898.012/0001-25,Dado não localizado,Ativa,"R BALACLAVA, 632, AP 3, JARDIM SANTO ALBERTO, ...",(11) 7789-7160,(11) 2669-0719,None,None,None


In [8]:
df_final.to_excel("telefones_separados.xlsx", index=False)